# Etap 1
Mam 9 plików tracking_week_*, każdy po prawie 1GB, reszta plików jest stosunkowo mniejsza. Dataset pozykany z zadania konkursowego z [Kaggle.](https://www.kaggle.com/competitions/nfl-big-data-bowl-2025/overview)

Połączyłam pliki `tracking_week_*` w jeden, dodając kolumnę `week` wskazującą na oryginalne pochodzenie danych.

In [ ]:
import dask.dataframe as dd
import glob


file_path = "nfl-big-data-bowl-2025/tracking/tracking_week_*.csv"
file_list = glob.glob(file_path)

dfs = []
for file in file_list:
    week_number = int(file.split("_week_")[1].split(".csv")[0])
    df = dd.read_csv(file, assume_missing=True)
    df['week'] = week_number
    dfs.append(df)

combined_ddf = dd.concat(dfs)

output_path = "project_data/tracking_combined.parquet"
combined_ddf.to_parquet(output_path, engine='pyarrow', write_index=False)

Były problemy z typami `bool` w pliku `player_play.csv`, dlatego przy jego konwersji narzucamy typy

In [ ]:
import dask.dataframe as dd

dtype = {
    "gameId": "int64",
    "playId": "int64",
    "nflId": "float64",
    "teamAbbr": "object",
    "hadRushAttempt": "float64",
    "rushingYards": "float64",
    "hadDropback": "float64",
    "passingYards": "float64",
    "sackYardsOffense": "float64",
    "hadPassReception": "float64",
    "receivingYards": "float64",
    "wasTargettedReceiver": "float64",
    "yardageGainedAfterTheCatch": "float64",
    "fumbles": "float64",
    "fumbleLost": "float64",
    "fumbleOutOfBounds": "float64",
    "assistedTackle": "float64",
    "forcedFumbleAsDefense": "float64",
    "halfSackYardsAsDefense": "float64",
    "passDefensed": "float64",
    "quarterbackHit": "float64",
    "sackYardsAsDefense": "float64",
    "safetyAsDefense": "float64",
    "soloTackle": "float64",
    "tackleAssist": "float64",
    "tackleForALoss": "float64",
    "tackleForALossYardage": "float64",
    "hadInterception": "float64",
    "interceptionYards": "float64",
    "fumbleRecoveries": "float64",
    "fumbleRecoveryYards": "float64",
    "wasInitialPassRusher": "float64",
    "penaltyNames": "object",
    "causedPressure": "float64",
    "timeToPressureAsPassRusher": "float64",
    "getOffAsPassRusher": "float64",
    "inMotionAtBallSnap": "float64",
    "shiftSinceLineset": "float64",
    "motionSinceLineset": "float64",
    "wasRunningRoute": "float64",
    "routeRan": "object",
    "blockedPlayerNFLId1": "float64",
    "blockedPlayerNFLId2": "float64",
    "blockedPlayerNFLId3": "float64",
    "pressureAllowedAsBlocker": "float64",
    "timeToPressureAllowedAsBlocker": "float64",
    "pff_defensiveCoverageAssignment": "object",
    "pff_primaryDefensiveCoverageMatchupNflId": "float64",
    "pff_secondaryDefensiveCoverageMatchupNflId": "float64",
}

p_df = dd.read_csv(
    'nfl-big-data-bowl-2025/player_play.csv',
    assume_missing=True,
    dtype=dtype
)
output_path = 'project_data/player_play.parquet'
p_df.to_parquet(output_path, engine='pyarrow', write_index=False)


# Etap 2

## Podstawowe statystyki

In [2]:
import dask.dataframe as dd

games = dd.read_csv('project_data/games.csv')
plays = dd.read_csv('project_data/plays.csv', assume_missing=True)
player_play = dd.read_parquet('project_data/player_play.parquet')
tracking = dd.read_parquet('project_data/tracking_combined.parquet')

games.describe().compute()

,gameId,season,week,homeFinalScore,visitorFinalScore
count,1.360000e+02,136.0,136.000000,136.000000,136.000000
mean,2.022099e+09,2022.0,4.845588,22.669118,20.948529
std,5.996614e+03,0.0,2.567254,8.659874,9.731446
min,2.022091e+09,2022.0,1.000000,3.000000,0.000000
25%,2.022093e+09,2022.0,3.000000,17.000000,14.750000
50%,2.022101e+09,2022.0,5.000000,22.500000,20.000000
75%,2.022102e+09,2022.0,7.000000,27.000000,27.000000
max,2.022111e+09,2022.0,9.000000,49.000000,48.000000


In [3]:
plays.describe().compute()

,gameId,playId,quarter,down,yardsToGo,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,absoluteYardlineNumber,preSnapHomeTeamWinProbability,...,timeInTackleBox,timeToSack,qbKneel,penaltyYards,prePenaltyYardsGained,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,pff_runPassOption
count,1.612400e+04,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,...,8917.000000,608.000000,16124.000000,384.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000
mean,2.022099e+09,2023.830563,2.572253,1.800794,8.461300,29.226185,11.255830,10.049615,60.413483,0.552007,...,2.632755,4.515796,0.010233,1.333333,5.475254,5.460618,-0.000065,0.000065,-0.029381,0.101340
std,5.979252e+03,1182.090771,1.135404,0.827194,3.908658,12.662828,9.468489,9.224680,24.326003,0.290603,...,0.817805,1.314407,0.100643,8.785422,8.622005,8.833095,0.041561,0.041561,1.397405,0.301787
min,2.022091e+09,54.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,11.000000,0.001049,...,0.000000,2.135000,0.000000,-20.000000,-18.000000,-68.000000,-0.834887,-0.541161,-13.023600,0.000000
25%,2.022093e+09,996.000000,2.000000,1.000000,6.000000,21.000000,3.000000,0.000000,40.000000,0.324613,...,2.079000,3.643750,0.000000,0.000000,0.000000,0.000000,-0.012808,-0.013166,-0.645440,0.000000
50%,2.022101e+09,2017.000000,3.000000,2.000000,10.000000,30.000000,10.000000,7.000000,61.000000,0.578922,...,2.569000,4.249000,0.000000,0.000000,3.000000,3.000000,0.000000,0.000000,-0.178783,0.000000
75%,2.022102e+09,3022.000000,4.000000,2.000000,10.000000,40.000000,17.000000,16.000000,81.000000,0.791601,...,3.100000,5.139000,0.000000,5.000000,8.000000,8.000000,0.013166,0.012808,0.696452,0.000000
max,2.022111e+09,5120.000000,5.000000,4.000000,38.000000,50.000000,49.000000,48.000000,109.000000,0.999197,...,7.400000,11.912000,1.000000,15.000000,98.000000,98.000000,0.541161,0.834887,8.698986,1.000000


In [4]:
player_play.describe().compute()

,gameId,playId,nflId,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,receivingYards,...,shiftSinceLineset,motionSinceLineset,wasRunningRoute,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
count,3.547270e+05,354727.000000,354727.000000,354727.000000,354727.000000,354727.000000,354727.000000,354727.000000,354727.000000,354727.000000,...,176178.000000,90238.000000,42779.0,49104.000000,4373.000000,7.000000,53044.000000,4080.000000,43484.000000,2387.000000
mean,2.022099e+09,2023.833091,47437.201033,0.019136,0.087304,0.017571,0.173268,-0.011471,0.015857,0.173373,...,0.021325,0.064518,1.0,46503.634694,49704.308027,49423.428571,0.076917,2.938971,47937.557078,47982.880184
std,5.979081e+03,1182.056487,5167.218156,0.137003,1.098183,0.131387,1.840473,0.309496,0.124923,1.841174,...,0.144466,0.245675,0.0,5105.945879,4056.385738,3275.463471,0.266463,0.989947,4696.183053,4603.065248
min,2.022091e+09,54.000000,25511.000000,0.000000,-10.000000,0.000000,-10.000000,-18.000000,0.000000,-11.000000,...,0.000000,0.000000,1.0,33131.000000,35454.000000,46269.000000,0.000000,0.800000,29550.000000,30842.000000
25%,2.022093e+09,996.000000,43426.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.0,43316.000000,46204.000000,47786.500000,0.000000,2.300000,44841.000000,44860.000000
50%,2.022101e+09,2017.000000,46457.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.0,46141.000000,48198.000000,47944.000000,0.000000,2.700000,47791.000000,46705.000000
75%,2.022102e+09,3022.000000,52590.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.0,52448.000000,53501.000000,50722.500000,0.000000,3.300000,52608.000000,52644.500000
max,2.022111e+09,5120.000000,55241.000000,1.000000,75.000000,1.000000,98.000000,0.000000,1.000000,98.000000,...,1.000000,1.000000,1.0,55241.000000,55239.000000,54733.000000,1.000000,11.600000,55168.000000,55157.000000


In [5]:
tracking.describe().compute()

,gameId,playId,nflId,frameId,jerseyNumber,x,y,s,a,dis,o,dir,week
count,5.932737e+07,5.932737e+07,5.674792e+07,5.932737e+07,5.674792e+07,5.932737e+07,5.932737e+07,5.932737e+07,5.932737e+07,5.932737e+07,5.675019e+07,5.675019e+07,5.932737e+07
mean,2.022099e+09,1.978953e+03,4.742374e+04,8.742606e+01,4.845992e+01,6.040085e+01,2.683161e+01,1.355132e+00,9.343592e-01,1.384822e-01,1.791824e+02,1.802452e+02,4.852460e+00
std,5.978099e+03,1.174310e+03,5.158914e+03,5.952667e+01,3.082830e+01,2.457768e+01,7.653872e+00,1.874221e+00,1.174222e+00,1.929432e-01,9.760752e+01,1.044449e+02,2.558991e+00
min,2.022091e+09,5.400000e+01,2.551100e+04,1.000000e+00,1.000000e+00,-1.179000e+01,-1.836000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.022093e+09,1.281000e+03,4.515900e+04,4.400000e+01,2.400000e+01,5.091000e+01,2.299000e+01,1.100000e-01,1.700000e-01,2.000000e-02,9.299000e+01,9.390000e+01,3.000000e+00
50%,2.022101e+09,2.534000e+03,4.787300e+04,8.700000e+01,5.700000e+01,7.288000e+01,2.745000e+01,6.700000e-01,5.700000e-01,7.000000e-02,1.870600e+02,1.849500e+02,5.000000e+00
75%,2.022102e+09,3.485000e+03,5.347600e+04,1.350000e+02,7.800000e+01,8.652000e+01,3.165000e+01,2.150000e+00,1.440000e+00,2.200000e-01,2.735100e+02,2.728700e+02,7.000000e+00
max,2.022111e+09,5.120000e+03,5.524100e+04,1.010000e+03,9.900000e+01,1.310100e+02,7.174000e+01,4.199000e+01,6.767000e+01,5.053000e+01,3.600000e+02,3.600000e+02,9.000000e+00


In [6]:
categorical_columns = ["frameType", "playDirection", "event"]
for col in categorical_columns:
    print(f"\nRozkład wartości dla kolumny {col}:")
    print(tracking[col].value_counts().compute())


Rozkład wartości dla kolumny frameType:
frameType
BEFORE_SNAP    38807440
SNAP             370852
AFTER_SNAP     20149081
Name: count, dtype: int64[pyarrow]

Rozkład wartości dla kolumny playDirection:
playDirection
left     29593364
right    29734009
Name: count, dtype: int64[pyarrow]

Rozkład wartości dla kolumny event:
event
autoevent_ballsnap              161
lateral                        1702
pass_outcome_touchdown         4186
play_action                   63135
ball_snap                    369426
huddle_break_offense         292307
qb_spike                        529
qb_strip_sack                  1311
touchback                       483
touchdown                      9936
line_set                     367517
pass_outcome_interception      4324
run                           20953
autoevent_passforward           552
fumble                         4761
fumble_offense_recovered       2898
qb_slide                       2760
run_pass_option                  23
man_in_motion        

## Preprocessing

Analiza typów danych, zmiana jeśli korzystne

Szukamy/wypełniamy puste wartości, sprawdzamy unikalność kolumn ID.

In [7]:
tracking.dtypes

gameId                   float64
playId                   float64
nflId                    float64
displayName      string[pyarrow]
frameId                  float64
frameType        string[pyarrow]
time             string[pyarrow]
jerseyNumber             float64
club             string[pyarrow]
playDirection    string[pyarrow]
x                        float64
y                        float64
s                        float64
a                        float64
dis                      float64
o                        float64
dir                      float64
event            string[pyarrow]
week                       int64
dtype: object

In [ ]:
tracking['nflId'].isnull().sum().compute()

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [13]:
og_mem = tracking.map_partitions(lambda df: df.memory_usage(deep=True).sum()).compute().sum()
print(f"Rozmiar danych przed zmianą typów: {og_mem / 1024**2:.2f} MB")

tracking_column_types = {
    "gameId": "int32",
    "playId": "int32",
    "nflId": "int32",
    "frameId": "int32",
    "jerseyNumber": "int32",
    "club": "category",
    "frameType": "category",
    "playDirection": "category",
    "event": "category",
    "x": "float32",
    "y": "float32",
    "s": "float32",
    "a": "float32",
    "dis": "float32",
    "o": "float32",
    "dir": "float32",
    "week": "int8",
}

tracking = tracking.astype(tracking_column_types)

new_mem = tracking.map_partitions(lambda df: df.memory_usage(deep=True).sum()).compute().sum()
print(f"Rozmiar danych po zmianie typów: {new_mem / 1024**2:.2f} MB")

Rozmiar danych przed zmianą typów: 11558.41 MB


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer: Error while type casting for column 'nflId'

In [8]:
games.dtypes

gameId                         int64
season                         int64
week                           int64
gameDate             string[pyarrow]
gameTimeEastern      string[pyarrow]
homeTeamAbbr         string[pyarrow]
visitorTeamAbbr      string[pyarrow]
homeFinalScore                 int64
visitorFinalScore              int64
dtype: object

In [9]:
plays.dtypes

gameId                                      float64
playId                                      float64
playDescription                     string[pyarrow]
quarter                                     float64
down                                        float64
yardsToGo                                   float64
possessionTeam                      string[pyarrow]
defensiveTeam                       string[pyarrow]
yardlineSide                        string[pyarrow]
yardlineNumber                              float64
gameClock                           string[pyarrow]
preSnapHomeScore                            float64
preSnapVisitorScore                         float64
playNullifiedByPenalty              string[pyarrow]
absoluteYardlineNumber                      float64
preSnapHomeTeamWinProbability               float64
preSnapVisitorTeamWinProbability            float64
expectedPoints                              float64
offenseFormation                    string[pyarrow]
receiverAlig

In [10]:
player_play.dtypes

gameId                                                  int64
playId                                                  int64
nflId                                                 float64
teamAbbr                                      string[pyarrow]
hadRushAttempt                                        float64
rushingYards                                          float64
hadDropback                                           float64
passingYards                                          float64
sackYardsAsOffense                                    float64
hadPassReception                                      float64
receivingYards                                        float64
wasTargettedReceiver                                  float64
yardageGainedAfterTheCatch                            float64
fumbles                                               float64
fumbleLost                                            float64
fumbleOutOfBounds                                     float64
assisted

In [11]:
missing_tracking = tracking.isnull().sum().compute()
missing_tracking

gameId                  0
playId                  0
nflId             2579451
displayName             0
frameId                 0
frameType               0
time                    0
jerseyNumber      2579451
club                    0
playDirection           0
x                       0
y                       0
s                       0
a                       0
dis                     0
o                 2577179
dir               2577179
event            56780860
week                    0
dtype: int64